# Assignment A1b - Functions and Computation
---
### Ryan Lin


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


## 1a sinewave

In [4]:
def sinewave(t, f=1.0, d=0.0):
    return np.sin(2 * np.pi * f * t + d)


## 1b gabor


spatial signal

In [5]:
def gabor(t, f=1.0, sigma=1.0, d=0.0, a=1.0):
    gaussian_envolope = np.exp(-t**2 / (2 * sigma**2))
    sinusoid = np.cos(2 * np.pi * f * t + d)
    return a * gaussian_envolope * sinusoid



In [ ]:
def gabore(t, f=1.0, sigma=1.0, a=1.0):
    return gabor(t, f, sigma, 0.0, a) # delay is 0 for cosine

def gaboro(t, f=1.0, sigma=1.0, a=1.0):
    return gabor(t, f, sigma, np.pi/2, a) # delay is pi/2 for sine

In [6]:
def gabor_norm(f=1.0, sigma=1.0, fs=1000):
    t = np.linspace(-3 * sigma, 3 * sigma, int(6 * sigma * fs))
    g = gabor(t, f, sigma, 0.0, 1.0)
    return 1 / np.sqrt(np.sum(g**2))

def gabore_norm(f=1.0, sigma=1.0, fs=1000):
    return gabor_norm(f, sigma, fs)

def gaboro_norm(f=1.0, sigma=1.0, fs=1000):
    t = np.linspace(-3 * sigma, 3 * sigma, int(6 * sigma * fs))
    g = gabor(t, f, sigma, np.pi/2, 1.0)
    return 1 / np.sqrt(np.sum(g**2))

## 1c gammatone

cochlear signal

In [11]:
def erb(f):
    return 24.7*((4.37 * f)/1000) + 1

def bandwidth(f):
    return 1.019 * erb(f)

def gammatone(t, f=1.0, n=4, d=0.0, a=1.0):
    t = np.maximum(t,0) # t >= 0
    b = bandwidth(f)
    return a * (t**(n-1)) * np.exp(-2 * np.pi * b * t) * sinewave(t, f, np.pi/2) #cosine is 2 pi

def gammatone_norm(f=1.0, n=4, fs=1000):
    t = np.linspace(0, 0.1, int(0.1 * fs))  # 0 to 0.1 seconds
    g = gammatone(t, f, n, 0.0, 1.0)
    return 1 / np.sqrt(np.sum(g**2))
